In [1]:
import pandas as pd

In [63]:
coso = pd.read_csv('coso.csv')
new0 = pd.read_csv('New_DB_new0.csv')

In [64]:
new0 = new0[['PP', 'slope', 'valor_humedad_suelo1', 'Latitud', 'Longitud']].dropna()
coso = coso[['PP', 'slope', 'valor_humedad_suelo1', 'Latitud', 'Longitud']].dropna()

In [65]:
data_0 = pd.read_csv('xterrae_data_0').dropna()
data_1 = pd.read_csv('xterrae_data_1').dropna()

In [66]:
new0

,PP,slope,valor_humedad_suelo1,Latitud,Longitud
0,71.563169,23.981049,0.438995,-39.597580,-72.260578
1,71.563169,26.626618,0.438995,-39.598738,-72.254661
2,71.563169,28.545852,0.438995,-39.582146,-72.258995
3,71.563169,28.545852,0.438995,-39.580483,-72.272591
4,7.532796,35.594937,0.438187,-39.561471,-72.286932
...,...,...,...,...,...
1355,1.167900,45.373746,0.352768,-39.615112,-73.198518
1356,1.054908,28.705555,0.313675,-39.663321,-73.166174
1357,0.000000,35.755171,0.306458,-39.556847,-73.134727
1358,0.000000,40.502730,0.335510,-39.460378,-73.158058


In [67]:
data_0 = pd.merge(new0, data_0, on=['Latitud', 'Longitud'], how='inner')
data_1 = pd.merge(coso, data_1, on=['Latitud', 'Longitud'], how='inner')

In [68]:
data_0 = data_0.drop(columns=['Unnamed: 0', 'Latitud', 'Longitud'])
data_1 = data_1.drop(columns=['Unnamed: 0', 'Latitud', 'Longitud'])
data_0['Valor'] = 0
data_1['Valor'] = 1

In [73]:
final_db = pd.read_csv('final_db.csv')
final_db = final_db.drop(columns='Unnamed: 0')

In [70]:
def comparar_columnas(df1, df2):
    # Obtener los nombres de las columnas en cada DataFrame
    columnas_df1 = set(df1.columns)
    columnas_df2 = set(df2.columns)
    
    # Encontrar columnas que están en df1 pero no en df2
    solo_en_df1 = columnas_df1 - columnas_df2
    
    # Encontrar columnas que están en df2 pero no en df1
    solo_en_df2 = columnas_df2 - columnas_df1
    
    # Imprimir los resultados
    if solo_en_df1:
        print("Columnas en df1 pero no en df2:")
        for columna in solo_en_df1:
            print(columna)
    else:
        print("Todas las columnas de df1 están en df2.")
    
    if solo_en_df2:
        print("Columnas en df2 pero no en df1:")
        for columna in solo_en_df2:
            print(columna)
    else:
        print("Todas las columnas de df2 están en df1.")

In [74]:
comparar_columnas(final_db,data_0)

Todas las columnas de df1 están en df2.
Todas las columnas de df2 están en df1.


In [75]:
comparar_columnas(final_db,data_1)

Todas las columnas de df1 están en df2.
Todas las columnas de df2 están en df1.


In [80]:
final = pd.concat([final_db, data_0])
final = pd.concat([final, data_1])

In [83]:
final = final.dropna()

In [84]:
final_x = final.drop(columns='Valor')
final_y = final.Valor

In [86]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

def cart_feature_selection(df, target_column, n_features=5):
    X = df.drop(target_column, axis=1)
    y = df[target_column]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    cart = RandomForestClassifier(random_state=42)
    #cart = XGBClassifier(random_state=42)
    cart.fit(X_train, y_train)
    
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': cart.feature_importances_
    }).sort_values('importance', ascending=False)
    
    selected_features = feature_importance['feature'][:n_features].tolist()
    
    return selected_features


In [90]:
cart_features = cart_feature_selection(final, 'Valor', n_features=50)

In [91]:
X = final[cart_features]
y = final.Valor

In [92]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBClassifier

param_grid = {
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'n_estimators': [100, 200, 300, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0, 0.1, 0.5, 1.0],
    'reg_lambda': [0, 1.0, 10.0]
}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(f'Numero de entrenamiendo: {X_train.shape}, Numero de test: {X_test.shape}')
model = XGBClassifier()
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=50, scoring='accuracy', cv=5, verbose=1, random_state=42)
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_true=y_test, y_pred=y_pred)
print(f'Precisión de {acc*100}')

Numero de entrenamiendo: (2518, 50), Numero de test: (630, 50)
Fitting 5 folds for each of 50 candidates, totalling 250 fits
